In [1]:
from __future__ import print_function
import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext, DataFrame
from pyspark.sql.functions import desc, mean

In [2]:
sc = SparkContext.getOrCreate()
# sc = SparkContext() 
sc

<SparkContext master=yarn appName=PySparkShell>

In [3]:
sqlContext = SQLContext(sc)
#df = sqlContext.read.csv('/data/ProjectDatasetTwitter/statuses.log.2014-12-04.gz') # about 15 minutes to run 
df = sqlContext.read.json('/data/ProjectDatasetTwitter/statuses.log.2014-12-04.gz')

In [4]:
print(df.show())

+------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------+---------+------------+----+------------------+------------------+-----------------------+---------------------+-------------------------+-------------------+-----------------------+----+-----+------------------+-------------+---------+--------------------+------+--------------------+--------------------+-------------+---------+--------------------+---------------------+
|contributors|coordinates|          created_at|              delete|            entities|   extended_entities|favorite_count|favorited|filter_level| geo|                id|            id_str|in_reply_to_screen_name|in_reply_to_status_id|in_reply_to_status_id_str|in_reply_to_user_id|in_reply_to_user_id_str|lang|place|possibly_sensitive|retweet_count|retweeted|    retweeted_status|scopes|              source|                text| timestamp_ms|truncated|                user|withheld_in_countries|
+-

In [5]:
df.show(n=20, truncate=False)

+------------+-----------+------------------------------+-------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
df.printSchema()

mylist = df.head(5) 
for item in mylist:
    print('row=', item, '\n')



root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- delete: struct (nullable = true)
 |    |-- status: struct (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- id_str: string (nullable = true)
 |    |    |-- user_id: long (nullable = true)
 |    |    |-- user_id_str: string (nullable = true)
 |    |-- timestamp_ms: string (nullable = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |  

In [9]:

import itertools
# int_num = df.count()
# print('total num=', int_num)
df.na.drop(subset=["entities"])

df.registerTempTable("tweets")
t = sqlContext.sql("SELECT distinct id,entities.hashtags FROM tweets").rdd


#related_hashtags= t.map(lambda t: map(lambda t0: t0[2], t[1])).map(lambda t: list(itertools.combinations(t, 2))).flatMap(lambda t: t).map(lambda x: ('\t'.join(unicode(i) for i in x), 1)).reduceByKey(lambda x,y: x+y).sortByKey(False).map(lambda x: '\t'.join(unicode(i) for i in x)).repartition(1)
related_hashtags= t.map(lambda t: map(lambda t0: t0[2].lower(), t[1])) \
                     .map(lambda t: list(itertools.combinations(t, 2))) \
                     .flatMap(lambda t: t) \
                     .map(lambda t: sorted(t)) \
                     .map(lambda x: '\t'.join(unicode(i) for i in x)) \
                     .map(lambda t: (t, 1)) \
                     .reduceByKey(lambda x,y: x+y) \
                     .filter(lambda t: t[1]>=min_occurs) \
                     .sortByKey(False) \
                     .map(lambda x: '\t'.join(unicode(i) for i in x)) \
                     .repartition(1)
print('here', related_hashtags)
## save stats from tweets to hdfs
related_hashtags.saveAsTextFile("%s/%s" % ("/tmp/tests/00", "related_hashtags"))


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 4 times, most recent failure: Lost task 0.3 in stage 9.0 (TID 21, data7201-node9.fabric.zones.eait.uq.edu.au, executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/hdp/current/spark2-client/python/pyspark/worker.py", line 177, in main
    process()
  File "/usr/hdp/current/spark2-client/python/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/hdp/current/spark2-client/python/pyspark/rdd.py", line 2423, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/hdp/current/spark2-client/python/pyspark/rdd.py", line 2423, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/hdp/current/spark2-client/python/pyspark/rdd.py", 